In [151]:
import numpy as np

# Returns a copy of data which is normalized around mean of spinal base point
def normalize_spine_base(data):
    b = np.copy(data)
    spine = np.zeros(3)
    for i in range(3):
       spine[i] = np.mean(b[:, i])
    m, n = b.shape
    for i in range(n):
       b[:, i] -= spine[i % 3]
    return b

# Pulls up torso skeleton data from a general skeleton file, and normalizes it using normalize_spine_data()
def load_skeleton_data(filename):
    data = np.loadtxt(filename, dtype='float', delimiter=',', skiprows=1, usecols=(9, 10, 11, 18, 19, 20, 27, 28, 29, 36, 37, 38, 45, 46, 47, 54, 55, 56, 63, 64, 65, 72, 73, 74, 81, 82, 83, 90, 91, 92, 99, 100, 101, 108, 109, 110, 189, 190, 191))
    return normalize_spine_base(data)

In [152]:
import os, random
from pathlib import Path

# Chosse a random skeleton file for a given gesture
def choose_random_skeleton(gesture):
    current_dir = Path('.')
    data_dir = current_dir / 'data' / gesture
    return random.choice([str(f) for f in list(data_dir.glob("*.txt")) if f.is_file()])


In [153]:
# Number of samples to be taken from each gesture class
tot_samples = 3

In [154]:
# Taking tot_samples number of samples from both gesture classes
x_files = [ choose_random_skeleton('head_nod') for i in range(tot_samples) ]
y_files = [ choose_random_skeleton('arms_move_down') for i in range(tot_samples) ]

In [155]:
from dtw import dtw

# Get a combined list of the skeleton files chosen earlier
combined_files = x_files + y_files

print("Samples chosen:\n")
for i in combined_files:
    print(i)
    
# Initializing the similarity matrix to be used for final comparison
s_matrix = np.zeros( len(combined_files) ** 2).reshape(len(combined_files), len(combined_files))

# Populating the similarity matrix with distance outputs from DTW
for i, fi in enumerate(combined_files):
    fi_data = load_skeleton_data(fi)
    for j, fj in enumerate(combined_files):
        fj_data = load_skeleton_data(fj)
        dist, cost, acc, path = dtw(fi_data, fj_data, dist=lambda a, b: np.linalg.norm(a - b))
        s_matrix[i, j] = dist
print()

print("Similarity matrix is:\n")
print(s_matrix)
print()

s_matrix_c = np.copy(s_matrix)

print("Similar files are:\n")
for i in range(len(s_matrix_c)):
    s_matrix_c[i,i] = float("inf")
    min_index = np.argmin(s_matrix_c[i])
    print(combined_files[i])
    print(combined_files[min_index])
    print()

Samples chosen:

data\head_nod\Gesture52_Session 20-Participant 39-Block 7_118670000_Skeleton.txt
data\head_nod\Gesture52_Session 20-Participant 40-Block 1_1220811621_Skeleton.txt
data\head_nod\Gesture52_Session 18-Participant 36-Block 4_271330000_Skeleton.txt
data\arms_move_down\Gesture968_Session 16-Participant 32-Block 3_1437000000_Skeleton.txt
data\arms_move_down\Gesture968_Session 6-Participant 12-Block 3_56330000_Skeleton.txt
data\arms_move_down\Gesture968_Session 3-Participant 5-Block 8_827038820_Skeleton.txt

Similarity matrix is:

[[ 0.          0.40058228  0.49669788  0.63684059  0.56747494  0.49062729]
 [ 0.40058228  0.          0.32842351  0.65249574  0.28888665  0.38517657]
 [ 0.49669788  0.32842351  0.          0.41364983  0.44772093  0.53090386]
 [ 0.63684059  0.65249574  0.41364983  0.          0.83721406  0.94935478]
 [ 0.56747494  0.28888665  0.44772093  0.83721406  0.          0.35078922]
 [ 0.49062729  0.38517657  0.53090386  0.94935478  0.35078922  0.        ]]

Si

In [156]:
def generate_playlist(s_matrix, combined_files):
    similar_playlist = open('similar.pls', 'w')
    similar_playlist.write('[playlist]\n\n')
    s_matrix_c = np.copy(s_matrix)
    for i in range(len(s_matrix_c)):
        s_matrix_c[i,i] = float("inf")
        min_index = np.argmin(s_matrix_c[i])
        similar_playlist.write('File' + str(2*i + 1) + '=' + str(combined_files[i])[:-13] + '.avi\n\n')
        similar_playlist.write('File' + str(2*i + 2) + '=' + str(combined_files[min_index])[:-13] + '.avi\n\n')
    similar_playlist.write('NumberOfEntries=' + str(len(s_matrix)+1) + '\n')
    similar_playlist.write('Version=2')
    similar_playlist.close()

In [157]:
generate_playlist(s_matrix, combined_files)